In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24870034
paper_name = 'nair_perrone_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table1.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 122 x 4


In [7]:
original_data.head()

,Open reading frame/gene name,Aβ42-GFP localization pattern,Respiratory deficiency,Human orthologue
0,Chromatin remodeling/histone exchange,NaN,NaN,NaN
1,CHD1,Cytosolic,No,CHD2
2,HIR1,Single small punctate,No,HIRA
3,HTA2,Single small punctate,No,H2AFX
4,SWC5,Single small punctate,No,CFDP1


In [8]:
original_data['genes'] = original_data['Open reading frame/gene name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                      Open reading frame/gene name  \
index_input                                          
0            Chromatin remodeling/histone exchange   
9                       Lipid metabolism/transport   
21                         Mitochondrial functions   
61                      Gene expression/regulation   
78                              Mitotic cell cycle   
83                           Methionine metabolism   
87                               Purine metabolism   
90                               Spindle pole body   
93                            Ubiquitin/proteasome   
97                              Bud-site selection   
100                            MAP kinase activity   
104                                 Others/unknown   

            Aβ42-GFP localization pattern Respiratory deficiency  \
index_input                                                        
0                                     NaN                    NaN   
9                                     N

In [12]:
original_data = original_data.loc[t,:]

In [13]:
original_data['data'] = 1

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(110, 1)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [15987]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,15987
data_type,value
orf,
YBL003C,1
YBL008W,1
YBL021C,1
YBL024W,1
YBL033C,1


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,15987
,data_type,value
gene_id,orf,
91,YBL003C,1
96,YBL008W,1
109,YBL021C,1
112,YBL024W,1
121,YBL033C,1


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      15987          
data_type       value    valuez
gene_id orf                    
91      YBL003C     1  6.437814
96      YBL008W     1  6.437814
109     YBL021C     1  6.437814
112     YBL024W     1  6.437814
121     YBL033C     1  6.437814

# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 24870034...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  2.46it/s]

Updating the data_modified_on field...
